In [1]:
from sqlalchemy import create_engine
import pandas as pd
from config import password
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Postgres username, password, and database name
POSTGRES_ADDRESS = 'localhost' 
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'postgres' 
POSTGRES_PASSWORD = password 
POSTGRES_DBNAME = 'AirQuality_DB'
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
    .format(username=POSTGRES_USERNAME, 
    password=POSTGRES_PASSWORD, 
    ipaddress=POSTGRES_ADDRESS, 
    port=POSTGRES_PORT,  
    dbname=POSTGRES_DBNAME)) 
# Create the connection
engine = create_engine(postgres_str) 
connection = engine.connect()

In [3]:
county_table = pd.read_sql_table('County', connection)  
date_year_table = pd.read_sql_table('DateYear', connection) 
year = pd.read_sql_table('year', connection)  
defining_parameter = pd.read_sql_table('Defining_Parameter', connection)  
sites_table = pd.read_sql_table('Sites', connection)  
aq_table = pd.read_sql_table('AirQuality', connection)   
census_table = pd.read_sql_table('CensusPopulation', connection)  

In [4]:
aq_date_year_merge_df = date_year_table.merge(aq_table,on='Date')
aq_date_year_merge_df

,Date,year,aq_unique_no,State_Name,county_code,AQI,Category,site_no,Defining_Parameter
0,2010-01-01,2010,0,California,1,72,Moderate,06-001-0007,PM2.5
1,2010-01-01,2010,365,California,5,18,Good,06-005-0002,Ozone
2,2010-01-01,2010,726,California,7,58,Moderate,06-007-4001,PM2.5
3,2010-01-01,2010,1091,California,9,19,Good,06-009-0001,Ozone
4,2010-01-01,2010,1456,California,11,79,Moderate,06-011-1002,PM2.5
...,...,...,...,...,...,...,...,...,...
200088,2020-10-30,2020,194778,California,65,40,Good,06-065-0500,PM10
200089,2020-10-31,2020,191472,California,31,103,Unhealthy for Sensitive Groups,06-031-0500,PM10
200090,2020-10-31,2020,193748,California,57,49,Good,06-057-0005,PM2.5
200091,2020-10-31,2020,194474,California,63,69,Moderate,06-063-1006,PM2.5


In [6]:
county_aq_yearly = aq_date_year_merge_df.groupby(['year','Defining_Parameter','county_code'])
total_records_per_county = county_aq_yearly['Defining_Parameter'].count()
aqi_score_cum = county_aq_yearly['AQI'].sum()
# county_code = county_aq_yearly['county_code'].first()

avg_aqi_per_county_year_parameter =   aqi_score_cum / total_records_per_county
avg_aqi_per_county_year_parameter

year  Defining_Parameter  county_code
2010  CO                  23              5.000000
                          25             39.000000
      NO2                 1              36.933333
                          13             31.181818
                          19             40.000000
                                           ...    
2020  PM2.5               105            32.837500
                          107            60.938272
                          111            67.153846
                          113            49.050000
      SO2                 23              0.000000
Length: 1743, dtype: float64

In [29]:
avg_aqi_per_county_year_parameter_df = pd.DataFrame({'Avg_AQI':avg_aqi_per_county_year_parameter}).reset_index()

avg_aqi_per_county_year_parameter_df

,year,Defining_Parameter,county_code,Avg_AQI
0,2010,CO,23,5.000000
1,2010,CO,25,39.000000
2,2010,NO2,1,36.933333
3,2010,NO2,13,31.181818
4,2010,NO2,19,40.000000
...,...,...,...,...
1738,2020,PM2.5,105,32.837500
1739,2020,PM2.5,107,60.938272
1740,2020,PM2.5,111,67.153846
1741,2020,PM2.5,113,49.050000


In [37]:
avg_aqi_per_county_year_df = pd.pivot_table(avg_aqi_per_county_year_parameter_df, values = 'Avg_AQI',index=['year', 'county_code'],
                    columns=['Defining_Parameter'], aggfunc=np.sum).fillna(0).reset_index()
avg_aqi_per_county_year_df.head()

Defining_Parameter,year,county_code,CO,NO2,Ozone,PM10,PM2.5,SO2
0,2010,1,0.0,36.933333,43.218978,0.0,51.525253,0.0
1,2010,5,0.0,0.000000,40.324100,0.0,0.000000,0.0
2,2010,7,0.0,0.000000,53.973485,0.0,55.910891,0.0
3,2010,9,0.0,0.000000,48.630225,0.0,40.129630,0.0
4,2010,11,0.0,0.000000,37.624183,29.0,40.452381,0.0


Index(['year', 'county_code', 'CO', 'NO2', 'Ozone', 'PM10', 'PM2.5', 'SO2'], dtype='object', name='Defining_Parameter')

In [40]:
aq_date_year_merge_pop_df = avg_aqi_per_county_year_df.merge(census_table,on=['county_code','year'])
aq_date_year_merge_pop_df

,year,county_code,CO,NO2,Ozone,PM10,PM2.5,SO2,census_unique_no,population
0,2010,1,0.0,36.933333,43.218978,0.000000,51.525253,0.0,0,1512986
1,2010,5,0.0,0.000000,40.324100,0.000000,0.000000,0.0,20,37886
2,2010,7,0.0,0.000000,53.973485,0.000000,55.910891,0.0,30,219949
3,2010,9,0.0,0.000000,48.630225,0.000000,40.129630,0.0,40,45468
4,2010,11,0.0,0.000000,37.624183,29.000000,40.452381,0.0,50,21437
...,...,...,...,...,...,...,...,...,...,...
531,2019,105,0.0,0.000000,0.000000,0.000000,22.368030,0.0,529,12285
532,2019,107,0.0,31.500000,75.884298,89.681818,62.818182,0.0,539,466195
533,2019,109,0.0,0.000000,45.849162,0.000000,0.000000,0.0,549,54478
534,2019,111,0.0,33.000000,52.313380,68.545455,52.184615,0.0,559,846006


In [41]:
census_table.count()

census_unique_no    580
county_code         580
year                580
population          580
dtype: int64

In [46]:
aq_date_year_merge_pop_table = aq_date_year_merge_pop_df.merge(county_table,on='county_code')
aq_date_year_merge_pop_table

,year,county_code,CO,NO2,Ozone,PM10,PM2.5,SO2,census_unique_no,population,county_name
0,2010,1,0.0,36.933333,43.218978,0.0,51.525253,0.0,0,1512986,Alameda
1,2011,1,0.0,34.850000,46.729927,0.0,53.581731,0.0,1,1530915,Alameda
2,2012,1,0.0,35.304348,42.933333,0.0,49.728395,91.0,2,1553764,Alameda
3,2013,1,0.0,36.846154,43.493976,0.0,55.182156,0.0,3,1579593,Alameda
4,2014,1,0.0,36.515152,45.361290,0.0,52.316384,0.0,4,1607792,Alameda
...,...,...,...,...,...,...,...,...,...,...,...
531,2012,3,0.0,0.000000,0.000000,0.0,40.208333,0.0,12,1110,Alpine
532,2013,3,0.0,0.000000,0.000000,0.0,63.437500,0.0,13,1128,Alpine
533,2014,3,0.0,0.000000,0.000000,0.0,24.580000,0.0,14,1080,Alpine
534,2015,3,0.0,0.000000,0.000000,0.0,107.225806,0.0,15,1077,Alpine


In [47]:
json_table = aq_date_year_merge_pop_table.to_json(orient ='table')
json_table

'{"schema":{"fields":[{"name":"index","type":"integer"},{"name":"year","type":"integer"},{"name":"county_code","type":"integer"},{"name":"CO","type":"number"},{"name":"NO2","type":"number"},{"name":"Ozone","type":"number"},{"name":"PM10","type":"number"},{"name":"PM2.5","type":"number"},{"name":"SO2","type":"number"},{"name":"census_unique_no","type":"integer"},{"name":"population","type":"integer"},{"name":"county_name","type":"string"}],"primaryKey":["index"],"pandas_version":"0.20.0"},"data":[{"index":0,"year":2010,"county_code":1,"CO":0.0,"NO2":36.9333333333,"Ozone":43.2189781022,"PM10":0.0,"PM2.5":51.5252525253,"SO2":0.0,"census_unique_no":0,"population":1512986,"county_name":"Alameda"},{"index":1,"year":2011,"county_code":1,"CO":0.0,"NO2":34.85,"Ozone":46.7299270073,"PM10":0.0,"PM2.5":53.5817307692,"SO2":0.0,"census_unique_no":1,"population":1530915,"county_name":"Alameda"},{"index":2,"year":2012,"county_code":1,"CO":0.0,"NO2":35.3043478261,"Ozone":42.9333333333,"PM10":0.0,"PM2.5